## Importation des bibliothèques
Les bibliothèques nécessaires sont importées au début. `requests` pour les requêtes HTTP, `BeautifulSoup` pour l'analyse du XML, `csv` pour la manipulation des fichiers CSV, `pandas` pour afficher un tableau d'échantillon.

In [1]:
import requests
import pandas as pd
import os

## Configuration des paramètres pour la requête API
dans cet exemple, on veut récupérer les auteurs affiliés à l'ENSA Nantes (identifiant OpenAlex : i4210107693) dont la signature contient la chaîne de caractère "Nantes", pour exclure tous les co-auteurs non nantais. On se limite également à l'année 2022. 

In [2]:
url = "https://api.openalex.org/works"
params = {
    "page": 1,
    "per_page": 200,
    "filter": 'authorships.institutions.lineage:i4210107693,raw_affiliation_strings.search:"Nantes",publication_year:2022'
}

## Définition de la fonction pour récupérer les données

In [3]:
# Définition de la fonction pour fetch les données de l'API
def fetch_data(url, params):
    data_to_export = []
    while True:
        response = requests.get(url, params=params)
        if response.status_code != 200:
            print("Erreur lors de la requête API :", response.status_code, response.text)
            break

        data = response.json()
        print("Page", params['page'], ":", len(data.get('results', [])), "résultats trouvés.")

        # Extraire les informations nécessaires
        for work in data.get('results', []):
            title = work.get('display_name', 'N/A')
            doi = work.get('doi', 'N/A')
            work_id = work.get('id', 'N/A')
            publication_year = work.get('publication_year', 'N/A')
            landing_page_url = work.get('primary_location', {}).get('landing_page_url', 'N/A')

            for authorship in work.get('authorships', []):
                raw_aff_strings = authorship.get('raw_affiliation_strings', [])
                # Affichage pour le débogage
                print("Examine l'affiliation :", raw_aff_strings)
                # Parcourir chaque affiliation dans la liste
                for raw_aff_string in raw_aff_strings:
                    if "Nantes" in raw_aff_string:
                        author_name = authorship.get('raw_author_name', 'N/A')
                        institutions = authorship.get('institutions', [])
                        affiliation = institutions[0].get('display_name', 'N/A') if institutions else 'N/A'
                        signature = raw_aff_string
                        entry = [title, doi, work_id, publication_year, landing_page_url, author_name, affiliation, signature]
                        data_to_export.append(entry)
                        print("Ajouté :", entry)  # Affichage pour le débogage

        # Vérifier si c'est la fin des résultats
        if len(data.get('results', [])) < params.get("per_page", 200):
            break  # Si moins de résultats que le maximum par page, c'est la fin
        else:
            params['page'] += 1  # Passer à la page suivante
    return data_to_export


## Appel de la fonction et collecte des données

In [4]:
extracted_data = fetch_data(url, params)

Page 1 : 17 résultats trouvés.
Examine l'affiliation : ['Nantes Université, ENSA Nantes, École Centrale Nantes, CNRS, AAU-CRENAU, UMR 1563, F-44000 Nantes, France']
Ajouté : ['THE WHOLE IS OTHER THAN THE SUM OF ITS PARTS: SENSIBILITY ANALYSIS OF 360° URBAN IMAGE SPLITTING', 'https://doi.org/10.5194/isprs-annals-v-4-2022-33-2022', 'https://openalex.org/W4280538011', 2022, 'https://doi.org/10.5194/isprs-annals-v-4-2022-33-2022', 'B. Beaucamp', 'École Centrale de Nantes', 'Nantes Université, ENSA Nantes, École Centrale Nantes, CNRS, AAU-CRENAU, UMR 1563, F-44000 Nantes, France']
Examine l'affiliation : ['Nantes Université, ENSA Nantes, École Centrale Nantes, CNRS, AAU-CRENAU, UMR 1563, F-44000 Nantes, France']
Ajouté : ['THE WHOLE IS OTHER THAN THE SUM OF ITS PARTS: SENSIBILITY ANALYSIS OF 360° URBAN IMAGE SPLITTING', 'https://doi.org/10.5194/isprs-annals-v-4-2022-33-2022', 'https://openalex.org/W4280538011', 2022, 'https://doi.org/10.5194/isprs-annals-v-4-2022-33-2022', 'T. Leduc', 'Écol

## Conversion des données en DataFrame

In [5]:
df = pd.DataFrame(extracted_data, columns=['Title', 'DOI', 'Work ID', 'Publication Year', 'Landing Page URL', 'Author Name', 'Affiliation', 'Signature'])

# Affichage du DataFrame pour vérifier les données

In [6]:
print(df.head())

                                               Title  \
0  THE WHOLE IS OTHER THAN THE SUM OF ITS PARTS: ...   
1  THE WHOLE IS OTHER THAN THE SUM OF ITS PARTS: ...   
2  THE WHOLE IS OTHER THAN THE SUM OF ITS PARTS: ...   
3  THE WHOLE IS OTHER THAN THE SUM OF ITS PARTS: ...   
4  Designing thermally sensitive public spaces: a...   

                                                 DOI  \
0  https://doi.org/10.5194/isprs-annals-v-4-2022-...   
1  https://doi.org/10.5194/isprs-annals-v-4-2022-...   
2  https://doi.org/10.5194/isprs-annals-v-4-2022-...   
3  https://doi.org/10.5194/isprs-annals-v-4-2022-...   
4      https://doi.org/10.1080/13574809.2022.2062312   

                            Work ID  Publication Year  \
0  https://openalex.org/W4280538011              2022   
1  https://openalex.org/W4280538011              2022   
2  https://openalex.org/W4280538011              2022   
3  https://openalex.org/W4280538011              2022   
4  https://openalex.org/W4293239148      

## Export du csv dans le dossier data

In [7]:
data_directory = '..\data'
if not os.path.exists(data_directory):
    os.makedirs(data_directory)

In [9]:
csv_file_path = os.path.join(data_directory, 'signatures_authors_ensa2022-openalex.csv')
df.to_csv(csv_file_path, index=False)
print(f"Data exported successfully to {csv_file_path}")

Data exported successfully to ..\data\signatures_authors_ensa2022-openalex.csv
